# Access _PLINDER_ data for training ML models

The goal of this tutorial is to provide a simple and simple hands-on demo for a new user to access _PLINDER_ dataset in prepparation for training machine learning models.

Here, we are going to demonstrate how to get the key input data:
- protein receptor fasta sequence
- small molecules ligand SMILES string
- access to linked _apo_ and _pred_ structure


In the process, we will show:
- How to download the _PLINDER_ data
- How to query _PLINDER_ index and splits to select relevant data using `plinder.core` API
- Extract task-specific data one might want to use for training a task-specific ML model, eg. one protein, one ligand
- How to use `plinder.core` API and `PlinderDataset` class to supply dataset inputs for `train` or `val` splits
- Load linked `apo` and `pred` structures
- Example how to create a simple diversity sampler based on cluster labels

## Download _PLINDER_
To download, run: `plinder_download --release 2024-06 --iteration v2 --yes` <br>
This will download and unpack all neccesary files. For more information on download check out [Dataset Tutorial](https://plinder-org.github.io/plinder/tutorial/dataset.html#getting-the-data)


## Loading _PLINDER_

We recommend users interact with the dataset using _PLINDER_ Python API.

To install the API run: ``pip install plinder[loader]``. If you are using zsh terminal, you will have to quote the package like ``"plinder[loader]"``

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations
import os
import pandas as pd
from plinder.core.scores import query_index

os.environ["PLINDER_OFFLINE"] = "true"

### Load _PLINDER_ index with selected columns from annotations table
For a full list with descriptions, please refer to [docs](https://plinder-org.github.io/plinder/dataset.html#annotation-tables-index).

In [2]:
# get plinder index with selected annotation columns specified
plindex = query_index(
    columns=["system_id", "ligand_id",
             "ligand_rdkit_canonical_smiles", "ligand_is_ion",
             "ligand_is_artifact", "system_num_ligand_chains",
             "system_num_neighboring_protein_chains"],
    filters=[
        ("system_type", "==", "holo"),
        ("system_num_neighboring_protein_chains", "<=", 5)
    ]
)

OSError: Project was not passed and could not be determined from the environment.

In [ ]:
plindex.head()

In [ ]:
# Display number of system neighboring protein chains
plindex.groupby("system_num_neighboring_protein_chains").system_id.count()

### Extracting specific data using _PLINDER_ annotations
As we can see just from the data tables above - a significant fraction of _PLINDER_ systems contain complex multi protein chain systems.

#### Task specific selection
If we would like to focus on single protein and single ligand systems for training, we can use the annotated columns to filter out systems that:
- contain only one protein chain
- only one ligand

Remember: In _PLINDER_ artifacts and (single atom) ions are also included in the index if they are part of the pocket.
- We can use columns `ligand_is_ion` and `ligand_is_artifact` to only select "proper" ligands.

Let's find out how many annotated ligands are "proper".

In [ ]:
# define "proper" ligands that are not ions or artifacts
plindex["ligand_is_proper"] = (
    ~plindex["ligand_is_ion"] & ~plindex["ligand_is_artifact"]
)

In [ ]:
plindex.groupby("ligand_is_proper").system_id.count()

#### User choice

The annotations table gives flexibility to choose the systems for training:
- One could strictly choose to use only the data that contains single protein single ligand systems
- Alternatively one could expand the number of systems to include systems containing single proper ligands, and optionally ignore the artifacts and ions in the pocket

Let's compare the numbers of such systems!

In [ ]:
# create mask for single receptor single ligand systems
systems_1p1l = (plindex["system_num_neighboring_protein_chains"] == 1) & (plindex["system_num_ligand_chains"] == 1)

# make count of these "proper" ligands per system
plindex["system_proper_num_ligand_chains"] = plindex.groupby("system_id")["ligand_is_proper"].transform("sum")

# create mask only for single receptor single "proper" ligand systems
systems_proper_1p1l = (plindex["system_num_neighboring_protein_chains"] == 1) & (plindex["system_proper_num_ligand_chains"] == 1) & plindex["ligand_is_proper"]

print(f"Number of single receptor single ligand systems: {sum(systems_1p1l)}")
print(f"Number of single receptor single \"proper\" ligand systems: {sum(systems_proper_1p1l)}")

As we can see - the second choice can provide up to 20% more data for training, however, the caveat is that some of the interactions made by artifacts or ions may influence the binding pose of the "proper" ligand. The user could come up with further strategies to filtering using annotations table or external tools, but this is beyond the scope of this tutorial.

### Loading splits

Now, after curating the systems of interest, let's have a look at the splits using _PLINDER_ API.

- How to use {mod}`plinder.core` API and class {class}`PlinderDataset` class to supply dataset inputs for `train` or `val` splits

In [ ]:
from plinder.core import get_split

#### Getting the splits

The `get_split` function provided the current _PLINDER_ split, the detailed description of this DataFrame is provide in the [dataset documentation](https://plinder-org.github.io/plinder/dataset.html#splits-splits), but for our practical purposes we are mostly interested in `system_id` and `split` that assigns each of our systems to a specific split category.

In [ ]:
# get the current plinder split
split_df = get_split()

In [ ]:
split_df.head()

Some specific method developers working on _flexible_ docking may also find handy the annotation column `system_has_apo_or_pred` indicating if the system has available `apo` or `pred` linked structures (see later).

In [ ]:
split_df.groupby(["split", "system_has_apo_or_pred"]).system_id.count()

For simplicity let's merge plindex and split DataFrames into one

In [ ]:
# merge to a single DataFrame
plindex_split = plindex.merge(split_df, on="system_id", how="left")

### Getting links to `apo` or `pred` structures
:::{currentmodule} plinder.core
:::

For users interested in including `apo` and `pred` structures in their workflow, all the information needed can be obtained from the function {func}`query_links`

In [ ]:
from plinder.core.scores import query_links

In [ ]:
links_df = query_links(
    columns=["reference_system_id", "id", "sort_score"],
)

Note the table is sorted by `sort_score` that is resolution for `apo`s and `plddt` for `pred`s. The `apo` or `pred` is specified in the additionally added `kind` column.

In [ ]:
links_df.head()

If a user wants to consider only one linked structure per system - we can easily drop duplicates, first sorting by `sort_score`. Using this priority score, `pred` structures will not be used unless there is no `apo` available. Alternative can be achieved by sorting with `ascending=False`, or filtering by `kind=="pred"` column.

In [ ]:
single_links_df = links_df.sort_values("sort_score", ascending=True).drop_duplicates("reference_system_id")
single_links_df.head()

Now that we have links to `apo` / `pred` structures, we can see how many of those are available for our single protein single ligand systems

In [ ]:
plindex_split[systems_1p1l].groupby(["split", "system_has_apo_or_pred"]).system_id.count()

In [ ]:
plindex_split_1p1l_links = plindex_split[systems_1p1l].merge(single_links_df, left_on="system_id", right_on="reference_system_id", how="left")

In [ ]:
# let's check how many systems have linked structures
plindex_split_1p1l_links['system_has_linked_apo_or_pred'] = ~plindex_split_1p1l_links.filename.isna()
plindex_split_1p1l_links.groupby(["split", "system_has_linked_apo_or_pred"]).system_id.count()

#### Selecting final dataset
Let's select only the set that has linked structures for flexible docking

In [ ]:
plindex_final_df = plindex_split_1p1l_links[
    (plindex_split_1p1l_links.system_has_linked_apo_or_pred) & (plindex_split_1p1l_links.split != "removed")
]
plindex_final_df.groupby(["split", "system_has_linked_apo_or_pred"]).system_id.count()

In [ ]:
plindex_final_df[["ligand_rdkit_canonical_smiles", "filename"]].iloc[0].filename

### Using _PLINDER_ API to load dataset by split

In [ ]:
from plinder.core.loader import get_model_input_files, PlinderDataset

In [ ]:
train_dataset = PlinderDataset(
    df=plindex_final_df,
    split="train",
    num_alternative_structures=2,
    file_paths_only=True,
)

:::{currentmodule} plinder.core.loader
:::

:::{note}
function {func}`get_model_input_files` accepts `split =` "train", "val" or "test"
:::

In [ ]:
sample_dataset = get_model_input_files(
    plindex_final_df,
    split = "val",
    max_num_sample = 10,
    num_alternative_structures = 1,
    )

note, if files not already available this downloads them to `~/.local/share/plinder/{PLINDER_RELEASE}/{PLINDER_ITERATION}` directory

In [ ]:
# Inspect data
sample_dataset[0]

### Using _PLINDER_ clusters in sampling

#### Define diversity sampler function
:::{currentmodule} plinder.core
:::

In general, diversity can be sampled using cluster information described [here](https://plinder-org.github.io/plinder/dataset.html#clusters-clusters). All cluster information can easily be added to `plindex` by calling the following function {func}`get_extended_plindex`. <br>
Here, we have provided an example of how one might use the function `get_diversity_samples` which is based on  `torch.utils.data.WeightedRandomSampler`. This example is meant for demonstration purposes and users are encouraged to come up with sampling strategy that suits their need. <br>
For this example, we are going to use the sample dversity based on the following parameters:
`metric="pli_qcov"`,  `threshold=70` and `cluster=communities`.

In [ ]:
from plinder.core.loader import  get_diversity_samples
from plinder.core import get_extended_plindex
from typing import Literal

# Example sampler function, this is for demonstration purposes,
# users are advised to write a sampler that suit their need
def get_diversity_samples(
    split_df: pd.DataFrame,
    split: Literal["train", "val", "test"] = "train",
    metric: str = "pli_qcov",
    threshold: int = 70,
    cluster_type: str = "communities"):

    from torch.utils.data import WeightedRandomSampler

    split_df = split_df[split_df.split == split]
    cluster_counts = (split_df[f"{metric}__{threshold}__{cluster_type}"]\
                      .value_counts().rename("cluster_count"))
    split_df = split_df.merge(
        cluster_counts,
        left_on=f"{metric}__{threshold}__{cluster_type}",
        right_index=True)
    split_df.reset_index(inplace=True)
    cluster_weights = 1.0 / split_df.cluster_count.values
    sampler = WeightedRandomSampler(
        weights=cluster_weights,
        num_samples=len(cluster_weights))
    sampler_index = [i for i in sampler]
    return split_df.loc[
        tuple(sampler_index),
        ("system_id", "split")].drop_duplicates()

In [ ]:
# Note, this takes a couple of minutes to run
plindex_extended = get_extended_plindex()
cluster_columns = ["system_id"] + [col for col in plindex_extended.columns \
                                   if ("communities" in col) or ("component" in col)]
plindex_with_clusters = plindex_final_df.merge(plindex_extended[cluster_columns], on="system_id", how="left")
sampled_df =  get_diversity_samples(split_df=plindex_with_clusters,
                                    metric="pli_qcov",
                                    threshold=70,
                                    cluster_type="communities")

The returned dataframe could then be passed to {func}`get_model_input_files` the same way `plindex_final_df` was used above.